# Github Activity Time Series Analysis

Assumptions:

1. On the dates there were no commits it is assumed that there are no commits

In [66]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime
from time import strptime
import plotly.graph_objects as go

# Data Acquisition

In [3]:
drive.mount('/content/drive')
#%cd drive/MyDrive

Mounted at /content/drive


In [25]:
os.getcwd()
os.listdir('/content/drive/MyDrive/Colab Notebooks')

['modules.py',
 'VAE',
 'VAE_CIFAR.ipynb',
 'Initial Setup.ipynb',
 'Untitled',
 'github_data.csv',
 'github-commit-analysis.ipynb',
 'github.npy',
 'github-commit-time-series.ipynb',
 'repo_cat.csv']

In [ ]:
github_df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/github_data.csv')

In [12]:
github_df.head()

,0,1,2,3,4,5,6
0,692bba578efb5e305c9b116568e5aad75b3fdbb3,Mortada Mehyar <mortada@users.noreply.github.com>,Wed Apr 21 12:27:07 2021 +0800,DOC: add example for plotting asymmetrical err...,pandas-dev/pandas,2021-04-21,2021
1,855696cde0ef5d80a7d4bd3f6a2940c5a2fecb3f,Patrick Hoefler <61934744+phofl@users.noreply....,Wed Apr 21 01:23:07 2021 +0200,Add keyword sort to pivot_table (#40954),pandas-dev/pandas,2021-04-21,2021
2,eaaefd140289a5103679ac6748567f724c7be56a,attack68 <24256554+attack68@users.noreply.gith...,Wed Apr 21 01:21:22 2021 +0200,ENH: `Styler.highlight_quantile` method (#40926),pandas-dev/pandas,2021-04-21,2021
3,aab87997058f3c74ba70286620ebe792ee4ef169,attack68 <24256554+attack68@users.noreply.gith...,Wed Apr 21 01:01:03 2021 +0200,ENH: add `decimal` and `thousands` args to `St...,pandas-dev/pandas,2021-04-21,2021
4,9c43cd7675d96174051e470de1f45e2bf7c9ebdc,Simon Hawkins <simonjayhawkins@gmail.com>,Tue Apr 20 23:58:18 2021 +0100,[ArrowStringArray] Use utf8_upper and utf8_low...,pandas-dev/pandas,2021-04-20,2021


In [13]:
github_df.describe()

,0,1,2,3,4,5,6
count,4336299,4336299,4336299,4335970,4336299,4336299,4336299
unique,4336299,89436,4268305,4217716,34,10825,40
top,692bba578efb5e305c9b116568e5aad75b3fdbb3,chromium-autoroll <chromium-autoroll@skia-publ...,Wed Nov 1 03:56:19 2017 +1000,Merge remote-tracking branch 'origin/master' i...,torvalds/linux,2020-01-14,2020
freq,1,36339,137,12753,998326,1880,420494


# Add a line to rename the column date to datetime here itself

# Data Pre-Processing

,commit,author,date,message,repo,date_1
count,4336299,4336299,4336299,4335970,4336299,4336299
unique,4336299,89436,4268305,4217716,34,10825
top,692bba578efb5e305c9b116568e5aad75b3fdbb3,chromium-autoroll <chromium-autoroll@skia-publ...,Wed Nov 1 03:56:19 2017 +1000,Merge remote-tracking branch 'origin/master' i...,torvalds/linux,2020-01-14
freq,1,36339,137,12753,998326,1880


In [ ]:
#Logic check module -tbd
check=github_df['date'][0].split()
check
datetime(2021,4,21).strftime("%Y-%m-%d")

##  Converting the long format date column to date and time columns


In [ ]:
#from datetime import datetime

def date_time(x):
    # Split the date string and parse the components
    day_name, month_name, day_num, time, year,_ = x['date'].split()

    month_num = strptime(month_name, '%b').tm_mon # Convert month name to month number
    dt = datetime(int(year), month_num, int(day_num))

    return dt.strftime("%Y-%m-%d")


In [ ]:
github_df['date_1']=github_df.apply(date_time,axis=1)

In [ ]:
github_df['date_1'].unique()
github_df['year']=pd.to_datetime(github_df['date_1']).dt.year

In [6]:
#np.save('/content/drive/MyDrive/Colab Notebooks/github.npy',github_df) #tbd
github_df=np.load('/content/drive/MyDrive/Colab Notebooks/github.npy',allow_pickle=True)
#github_df.head(1)

In [7]:
github_df=pd.DataFrame(github_df)

In [8]:
github_df.head(1)

,0,1,2,3,4,5,6
0,692bba578efb5e305c9b116568e5aad75b3fdbb3,Mortada Mehyar <mortada@users.noreply.github.com>,Wed Apr 21 12:27:07 2021 +0800,DOC: add example for plotting asymmetrical err...,pandas-dev/pandas,2021-04-21,2021


In [9]:
# tbd
github_df=github_df.rename(columns={0:'commit',1:'author',2:'datetime',3:'message',4:'repo',5:'date',6:'year'})

In [109]:
github_df_repo=github_df.groupby(['date','year','repo']).agg({'commit':'count','author':'nunique'}).reset_index()
github_df_repo.head(2)

,date,year,repo,commit,author
0,1970-01-01,1970,torvalds/linux,1,1
1,1972-07-18,1972,golang/go,1,1


In [110]:
github_df_repo['commit'].sum() # The sum is matching with the total records . There are no duplicates
github_df_repo.shape

(150692, 5)

Removing the records where the recorded dates are in the future (three records in the future hence removed) or in the past very few instances of commit data is available

year
1970         1
1972         1
1974         1
1988         3
1989         4


Analysis is performed for years after 1989 for the sake of continutity and enough history is already available

In [111]:
date_check=github_df_repo.groupby(['year','repo'])['commit'].sum().reset_index()
date_check.tail()

,year,repo,commit
546,2021,torvalds/linux,12693
547,2021,v8/v8,2160
548,2030,torvalds/linux,1
549,2037,torvalds/linux,1
550,2085,torvalds/linux,1


In [112]:
github_df_repo.groupby(['year'])['commit'].sum() .head()

year
1970    1
1972    1
1974    1
1988    3
1989    4
Name: commit, dtype: int64

In [113]:
# Filtering the dataframe to retain complete data
github_df_repo=github_df_repo[(github_df_repo['year']>1989) & (github_df_repo['year']<2023)]
github_df_repo.shape

(150680, 5)

12 records were removed in total by filtering on years

## At repository level checking for the conitnuity of the data


In [114]:
def process_repo_day(github_repo):
  """
    github_repo: filtered github activity for a specific repo

    Create pandas date range based on the dates registered

    Merge the newly created continuous date index

  """

  all_dates = pd.date_range(start=github_repo['date'].min(),
                          end=github_repo['date'].max(), freq='D')
  new_index = pd.MultiIndex.from_product([all_dates,github_repo['repo'].unique()], names=['date', 'repo'])
# Reindex the DataFrame with the new MultiIndex
  full_df=pd.DataFrame(index=new_index).reset_index()
  github_repo['date']=pd.to_datetime(github_repo['date'])
  github_repo = pd.merge(full_df,github_repo, on=['date', 'repo'], how='left')
  github_repo.sort_values(by=['repo', 'date'], inplace=True)
 # github_df_repo_all=github_df_repo_all.drop(columns=['year'])
  github_repo['commit']=github_repo['commit'].fillna(0)
  github_repo['author']=github_repo['author'].fillna(0)
  return github_repo



In [102]:
lst_repo= github_df_repo['repo'].unique()
lst_repo # Displaying unique repositories

array(['gcc-mirror/gcc', 'python/cpython', 'freebsd/freebsd-src',
       'openbsd/src', 'apache/httpd', 'postgres/postgres', 'scipy/scipy',
       'llvm/llvm-project', 'chromium/chromium', 'torvalds/linux',
       'numpy/numpy', 'nginx/nginx', 'matplotlib/matplotlib',
       'ipython/ipython', 'tidyverse/ggplot2', 'golang/go', 'v8/v8',
       'nodejs/node', 'pandas-dev/pandas', 'scikit-learn/scikit-learn',
       'apache/spark', 'opencv/opencv', 'rust-lang/rust', 'apple/swift',
       'rstudio/rstudio', 'pytorch/pytorch', 'facebook/react',
       'kubernetes/kubernetes', 'microsoft/TypeScript', 'angular/angular',
       'jupyterlab/jupyterlab', 'tensorflow/tensorflow',
       'microsoft/vscode', 'denoland/deno'], dtype=object)

In [74]:
github_repo=github_df_repo.loc[github_df_repo['repo']=='tensorflow/tensorflow']

In [115]:
github_df_repo_all=pd.DataFrame()

for i in lst_repo:
  github_repo=github_df_repo.loc[github_df_repo['repo']==i]
  github_df_repo_all=github_df_repo_all.append(process_repo_day(github_repo),ignore_index=True)

<ipython-input-114-a1ba550fe3a6>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-115-bb660ae60bf1>:5: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-114-a1ba550fe3a6>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-115-bb660ae60bf1>:5: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipy

## Previous approach has to be removed

In [106]:
all_dates = pd.date_range(start=github_df_repo['date'].min(),
                          end=github_df_repo['date'].max(), freq='D')
# Creating the cross product of repo and dates
new_index = pd.MultiIndex.from_product([all_dates,github_df_repo['repo'].unique()], names=['date', 'repo'])
# Reindex the DataFrame with the new MultiIndex
full_df=pd.DataFrame(index=new_index).reset_index()
github_df_repo['date']=pd.to_datetime(github_df_repo['date'])
# Merge the original DataFrame with the full DataFrame to fill in missing dates
github_df_repo_all = pd.merge(full_df,github_df_repo, on=['date', 'repo'], how='left')
github_df_repo_all.head()
# Sort the result DataFrame by date and product
github_df_repo_all.sort_values(by=['repo', 'date'], inplace=True)


In [116]:
github_df_repo_all['commit'].sum()  # The join was successful

4336286.0

In [117]:
github_df_repo_all=github_df_repo_all.drop(columns=['year'])
github_df_repo_all['commit']=github_df_repo_all['commit'].fillna(0)
github_df_repo_all.head()

,date,repo,commit,author
0,1990-01-31,gcc-mirror/gcc,1.0,1.0
1,1990-02-01,gcc-mirror/gcc,0.0,0.0
2,1990-02-02,gcc-mirror/gcc,0.0,0.0
3,1990-02-03,gcc-mirror/gcc,0.0,0.0
4,1990-02-04,gcc-mirror/gcc,0.0,0.0


Analysing the repos with top commits along with the categorisation based on the utility of the repos

In [118]:
rep_cat_df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/repo_cat.csv')
rep_cat_df.head()

github_df_repo_all= pd.merge(github_df_repo_all,rep_cat_df,how='left',on='repo')

# Exploratory Data Analysis

Creating a summary table to analyse the length of data available along with the number of commits

In [107]:
github_df.loc[github_df['repo']=='torvalds/linux']['author'].nunique()

30109

In [133]:
commit_sum=github_df_repo_all.groupby(['repo','categorisation']).agg({'date':['min','max'],'commit':'sum'})
commit_sum=pd.DataFrame(commit_sum).reset_index()
#
commit_sum.columns = commit_sum.columns.map(''.join)
commit_sum.columns


Index(['repo', 'categorisation', 'datemin', 'datemax', 'commitsum'], dtype='object')

In [134]:
from dateutil import relativedelta
delta = commit_sum.apply(lambda row: relativedelta.relativedelta(row['datemax'],row['datemin']),axis=1)
commit_sum['years'] = delta.apply(lambda x: x.years)
commit_sum['months'] = delta.apply(lambda x: x.months)
#commit_sum.columns = commit_sum.columns.to_flat_index()
commit_sum.sort_values(by=['commitsum','years'],inplace=True,ascending=False)
commit_sum.head()


,repo,categorisation,datemin,datemax,commitsum,years,months
31,torvalds/linux,Operating Systems,2001-09-17,2021-04-20,998322.0,19,7
4,chromium/chromium,Web Browsers and Engines,2001-08-24,2021-04-21,995884.0,19,7
13,llvm/llvm-project,Compilers and Languages,2001-06-06,2021-04-21,386114.0,19,10
7,freebsd/freebsd-src,Operating Systems,1993-06-12,2021-04-21,258157.0,27,10
20,openbsd/src,Operating Systems,1995-10-18,2021-04-21,212189.0,25,6


In [121]:
github_df_cpython=github_df_repo_all.loc[github_df_repo_all['repo']=='python/cpython']
# Create a line plot
fig = go.Figure(data=go.Scatter(x=github_df_cpython['date'], y=github_df_cpython['commit'], mode='lines'))

# Set x and y-axis labels
fig.update_layout(xaxis_title='Day', yaxis_title='Github commit activity for python/cpython')

# Show the plot
fig.show()


Observations on the time series:
*   Change in the mean
*   Presence of **outliers**


*   Longest Time Series
*   






In [130]:
commit_sum['categorisation'].unique()

array(['Operating Systems', 'Web Browsers and Engines',
       'Compilers and Languages', 'Python Libraries',
       'Frameworks and Tools', 'R Libraries'], dtype=object)

In [132]:
# List of repositories to plot
#repos = ['python/cpython', 'another_repo', 'yet_another_repo']  # replace with your actual repo names

# Create an empty figure
fig = go.Figure()

lst_repo=commit_sum[commit_sum['categorisation']=='Operating Systems']['repo'].unique()

# Loop over the repositories and add a line for each one
for repo in lst_repo:
    data = github_df_repo_all.loc[github_df_repo_all['repo'] == repo]
    fig.add_trace(go.Scatter(x=data['date'], y=data['commit'], mode='lines', name=repo))

# Set x and y-axis labels
fig.update_layout(xaxis_title='Day', yaxis_title='Github commit activity')

# Show the plot
fig.show()


In [122]:
#fig, ax = plt.subplots(figsize=(10, 6))
#sns.lineplot(x='date',y='commit', data=github_df_repo_all.loc[github_df_repo_all['repo']=='torvalds/linux'])
data=github_df_repo_all.loc[github_df_repo_all['repo']=='gcc-mirror/gcc']
fig = go.Figure(data=go.Scatter(x=data['date'], y=data['commit'], mode='lines'))

# Set x and y-axis labels
fig.update_layout(xaxis_title='Day', yaxis_title='Github commit activity for torvalds/linux')

# Show the plot
fig.show()


In [135]:
from plotly.subplots import make_subplots
data=github_df_repo_all.loc[github_df_repo_all['repo']=='python/cpython']
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add a trace for the primary y-axis
fig.add_trace(
    go.Scatter(x=data['date'], y=data['commit'], mode='lines'),
    secondary_y=False,
)

# Add a trace for the secondary y-axis
# Replace 'secondary_data' with your actual secondary data
fig.add_trace(
    go.Scatter(x=data['date'], y=data['author'], mode='lines'),
    secondary_y=True,
)

# Set x-axis title
fig.update_xaxes(title_text='Day')

# Set y-axes titles
fig.update_yaxes(title_text='Github commit activity for torvalds/linux', secondary_y=False)
fig.update_yaxes(title_text='Number of authors committing', secondary_y=True)

# Show the plot
fig.show()
